# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 12 2022 10:09AM,246134,19,60015844,"GUSA Granules USA, Inc.",Released
1,Sep 12 2022 10:02AM,246132,19,8516620,"Graystone, LLC",Released
2,Sep 12 2022 9:58AM,246060,21,626339,"NBTY Global, Inc.",Released
3,Sep 12 2022 9:57AM,246127,15,8515427,"Carwin Pharmaceutical Associates, LLC",Released
4,Sep 12 2022 9:57AM,246127,15,8516582,"Carwin Pharmaceutical Associates, LLC",Released
5,Sep 12 2022 9:55AM,246100,21,626358,"NBTY Global, Inc.",Executing
6,Sep 12 2022 9:55AM,246126,15,8515479,"Brookfield Pharmaceuticals, LLC",Released
7,Sep 12 2022 9:55AM,246126,15,8515497,"Brookfield Pharmaceuticals, LLC",Released
8,Sep 12 2022 9:55AM,246126,15,8515498,"Brookfield Pharmaceuticals, LLC",Released
9,Sep 12 2022 9:55AM,246126,15,8515499,"Brookfield Pharmaceuticals, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
23,246128,Executing,1
24,246129,Released,14
25,246130,Released,4
26,246132,Released,1
27,246134,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
246128,1.0,NaN
246129,NaN,14.0
246130,NaN,4.0
246132,NaN,1.0
246134,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
246060,0.0,1.0
246096,0.0,1.0
246100,1.0,0.0
246103,6.0,1.0
246106,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
246060,0,1
246096,0,1
246100,1,0
246103,6,1
246106,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,246060,0,1
1,246096,0,1
2,246100,1,0
3,246103,6,1
4,246106,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,246060,,1
1,246096,,1
2,246100,1,
3,246103,6,1
4,246106,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 12 2022 10:09AM,246134,19,"GUSA Granules USA, Inc."
1,Sep 12 2022 10:02AM,246132,19,"Graystone, LLC"
2,Sep 12 2022 9:58AM,246060,21,"NBTY Global, Inc."
3,Sep 12 2022 9:57AM,246127,15,"Carwin Pharmaceutical Associates, LLC"
5,Sep 12 2022 9:55AM,246100,21,"NBTY Global, Inc."
6,Sep 12 2022 9:55AM,246126,15,"Brookfield Pharmaceuticals, LLC"
35,Sep 12 2022 9:55AM,246130,10,"Methapharm, Inc."
39,Sep 12 2022 9:55AM,246129,10,"Methapharm, Inc."
53,Sep 12 2022 9:54AM,246128,19,"GUSA Granules USA, Inc."
54,Sep 12 2022 9:52AM,246125,20,Alumier Labs Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Sep 12 2022 10:09AM,246134,19,"GUSA Granules USA, Inc.",,1
1,Sep 12 2022 10:02AM,246132,19,"Graystone, LLC",,1
2,Sep 12 2022 9:58AM,246060,21,"NBTY Global, Inc.",,1
3,Sep 12 2022 9:57AM,246127,15,"Carwin Pharmaceutical Associates, LLC",,2
4,Sep 12 2022 9:55AM,246100,21,"NBTY Global, Inc.",1,
5,Sep 12 2022 9:55AM,246126,15,"Brookfield Pharmaceuticals, LLC",,29
6,Sep 12 2022 9:55AM,246130,10,"Methapharm, Inc.",,4
7,Sep 12 2022 9:55AM,246129,10,"Methapharm, Inc.",,14
8,Sep 12 2022 9:54AM,246128,19,"GUSA Granules USA, Inc.",1,
9,Sep 12 2022 9:52AM,246125,20,Alumier Labs Inc.,,7


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 12 2022 10:09AM,246134,19,"GUSA Granules USA, Inc.",1,
1,Sep 12 2022 10:02AM,246132,19,"Graystone, LLC",1,
2,Sep 12 2022 9:58AM,246060,21,"NBTY Global, Inc.",1,
3,Sep 12 2022 9:57AM,246127,15,"Carwin Pharmaceutical Associates, LLC",2,
4,Sep 12 2022 9:55AM,246100,21,"NBTY Global, Inc.",,1
5,Sep 12 2022 9:55AM,246126,15,"Brookfield Pharmaceuticals, LLC",29,
6,Sep 12 2022 9:55AM,246130,10,"Methapharm, Inc.",4,
7,Sep 12 2022 9:55AM,246129,10,"Methapharm, Inc.",14,
8,Sep 12 2022 9:54AM,246128,19,"GUSA Granules USA, Inc.",,1
9,Sep 12 2022 9:52AM,246125,20,Alumier Labs Inc.,7,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 12 2022 10:09AM,246134,19,"GUSA Granules USA, Inc.",1,
1,Sep 12 2022 10:02AM,246132,19,"Graystone, LLC",1,
2,Sep 12 2022 9:58AM,246060,21,"NBTY Global, Inc.",1,
3,Sep 12 2022 9:57AM,246127,15,"Carwin Pharmaceutical Associates, LLC",2,
4,Sep 12 2022 9:55AM,246100,21,"NBTY Global, Inc.",,1


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Sep 12 2022 10:09AM,246134,19,"GUSA Granules USA, Inc.",1.0,NaN
1,Sep 12 2022 10:02AM,246132,19,"Graystone, LLC",1.0,NaN
2,Sep 12 2022 9:58AM,246060,21,"NBTY Global, Inc.",1.0,NaN
3,Sep 12 2022 9:57AM,246127,15,"Carwin Pharmaceutical Associates, LLC",2.0,NaN
4,Sep 12 2022 9:55AM,246100,21,"NBTY Global, Inc.",NaN,1.0


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 12 2022 10:09AM,246134,19,"GUSA Granules USA, Inc.",1.0,0.0
1,Sep 12 2022 10:02AM,246132,19,"Graystone, LLC",1.0,0.0
2,Sep 12 2022 9:58AM,246060,21,"NBTY Global, Inc.",1.0,0.0
3,Sep 12 2022 9:57AM,246127,15,"Carwin Pharmaceutical Associates, LLC",2.0,0.0
4,Sep 12 2022 9:55AM,246100,21,"NBTY Global, Inc.",0.0,1.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2461185,201.0,77.0
12,246113,2.0,0.0
15,738377,34.0,0.0
19,984497,3.0,7.0
20,492231,8.0,0.0
21,738256,2.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2461185,201.0,77.0
1,12,246113,2.0,0.0
2,15,738377,34.0,0.0
3,19,984497,3.0,7.0
4,20,492231,8.0,0.0
5,21,738256,2.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,201.0,77.0
1,12,2.0,0.0
2,15,34.0,0.0
3,19,3.0,7.0
4,20,8.0,0.0
5,21,2.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,201.0
1,12,Released,2.0
2,15,Released,34.0
3,19,Released,3.0
4,20,Released,8.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,19,20,21
Status,,,,,,
Executing,77.0,0.0,0.0,7.0,0.0,1.0
Released,201.0,2.0,34.0,3.0,8.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,19,20,21
0,Executing,77.0,0.0,0.0,7.0,0.0,1.0
1,Released,201.0,2.0,34.0,3.0,8.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,19,20,21
0,Executing,77.0,0.0,0.0,7.0,0.0,1.0
1,Released,201.0,2.0,34.0,3.0,8.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()